In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install transformers

     |████████████████████████████████| 1.8MB 8.3MB/s 
     |████████████████████████████████| 890kB 57.3MB/s 
     |████████████████████████████████| 3.2MB 56.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=90ebdb9a825cb2832e683614f306d2a951f4cb9da4769f5fed3c3e6bf332ede3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import random
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import Parameter
from transformers import BertModel
from transformers import BertConfig
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import torch.optim as optim
import copy
import matplotlib.pyplot as plt
from transformers import BertLayer
import pickle

## Load Data

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_df(file):
    return pd.read_csv(file, sep='|',header = None)

In [ ]:
train_df_fake = get_df('/content/drive/My Drive/WeiboRumorSet/WeiboRumorSet/tweets/train_rumor.txt')
train_df_real = get_df('/content/drive/My Drive/WeiboRumorSet/WeiboRumorSet/tweets/train_nonrumor.txt')
test_df_fake = get_df('/content/drive/My Drive/WeiboRumorSet/WeiboRumorSet/tweets/test_rumor.txt')
test_df_real = get_df('/content/drive/My Drive/WeiboRumorSet/WeiboRumorSet/tweets/test_nonrumor.txt')

In [ ]:
train_fake = train_df_fake[0].tolist()
train_real = train_df_real[0].tolist()
test_fake = test_df_fake[0].tolist()
test_real = test_df_real[0].tolist()

In [ ]:
def fix_offset(list_):
    fixed_flag = False

    while not fixed_flag:
        exit_flag=False
        temp = copy.deepcopy(list_)
        for i,v in enumerate(temp):
            if v!=None:
                if 'sinaimg.cn' in v:
                    if list_[i+1] !=None:
                        if list_[i+1].isdigit():
                            list_.insert(i+1,None)
                            exit_flag=True
                            break
        if not exit_flag:
            fixed_flag=True
            
    return list_

In [ ]:
train_fake = fix_offset(train_fake) 
train_real = fix_offset(train_real)
test_fake = fix_offset(test_fake)
test_real = fix_offset(test_real)

In [ ]:
def break_in_block(list_):
    temp = []
    for i in range(0,len(list_),3):
        temp.append(list_[i:i+3])
    return temp

In [ ]:
train_fake = break_in_block(train_fake)
train_real = break_in_block(train_real)
test_fake = break_in_block(test_fake)
test_real = break_in_block(test_real)

In [ ]:
def get_image_and_text_list(blocks_list):
    image_list = []
    text_list = []
    for i in blocks_list:
        if i[-1] !=None:
            image_list.append(i[1])
            text_list.append(i[-1])
    image_list = [i.split('/')[-1] for i in image_list]
    return image_list, text_list

In [ ]:
train_fake_image,train_fake_text = get_image_and_text_list(train_fake)
train_real_image,train_real_text = get_image_and_text_list(train_real)
test_fake_image,test_fake_text = get_image_and_text_list(test_fake)
test_real_image,test_real_text = get_image_and_text_list(test_real)

In [ ]:
train_fake_Y = [0]*len(train_fake_image)
train_real_Y = [1]*len(train_real_image)
test_fake_Y = [0]*len(test_fake_image)
test_real_Y = [1]*len(test_real_image)

In [ ]:
train_images = train_fake_image+train_real_image
train_text = train_fake_text + train_real_text
trainY = train_fake_Y+train_real_Y

test_images = test_fake_image+test_real_image
test_text = test_fake_text+test_real_text
testY = test_fake_Y+test_real_Y

In [ ]:
len(train_images),len(train_text),len(trainY),len(test_images),len(test_text),len(testY)

(7481, 7481, 7481, 1930, 1930, 1930)

In [ ]:
train_images = np.array(train_images)
train_text = np.array(train_text)
trainY = np.array(trainY)
test_images = np.array(test_images)
test_text = np.array(test_text)
testY = np.array(testY)

In [ ]:
from os import walk
def get_total_images() :
  mypath = "/content/drive/My Drive/multi-modal/weibo/airsplay/feat_rumor/"
  mypath1 = "/content/drive/My Drive/multi-modal/weibo/airsplay/feat_nonrumor/"
  img_feat = []
  for (dirpath, dirnames, filenames) in walk(mypath):
      img_feat.extend(filenames)
      break
  for (dirpath, dirnames, filenames) in walk(mypath1):
      img_feat.extend(filenames)
      break 
  mypath3 = "/content/drive/My Drive/WeiboRumorSet/WeiboRumorSet/rumor_images/"
  images = []
  for (dirpath, dirnames, filenames) in walk(mypath3):
      images.extend(filenames)
      break
  mypath4 = "/content/drive/My Drive/WeiboRumorSet/WeiboRumorSet/nonrumor_images/"
  for (dirpath, dirnames, filenames) in walk(mypath4):
      images.extend(filenames)
      break
  img_dict = {}
  for img in images :
    if (img[:-4] not in img_dict) :
      img_dict[img[:-4]] = img
    elif ((img[:-4] in img_dict) and img[-4:]=='.gif') :
      img_dict[img[:-4]] = img 
  total_img = []
  for img in img_feat :
    total_img.append(img_dict[img[:-4]])
  return total_img, images

In [ ]:
total_images, images = get_total_images()

Read feat rumor
Read feat nonrumor
Read feat rumor
Read feat non rumor


In [ ]:
def index_to_delete(list_, total_images):
    gif_list = ['957e1cf2tw1e5foxts295g206o03p4qp.gif','a716fd45jw1ev0cgf8j46g209505zh4i.gif','005vnhZYgw1evupo8ttddg308w06o4qp.gif','7da75521gw1ele2jvi85rg2096056u0x.gif']
    index = []
    for i,v in enumerate(list_):
        if v[:-4] not in total_images:
            index.append(i)
    return index

In [ ]:
train_delete_index =index_to_delete(train_images, t_img)
test_delete_index = index_to_delete(test_images, t_img)

4143

In [ ]:
train_images = np.delete(train_images,train_delete_index)
train_text = np.delete(train_text,train_delete_index)
trainY = np.delete(trainY,train_delete_index)
test_images = np.delete(test_images,test_delete_index)
test_text = np.delete(test_text,test_delete_index)
testY = np.delete(testY,test_delete_index)

In [ ]:
shuffle_index= np.arange(len(train_images))
np.random.shuffle(shuffle_index)
train_images = train_images[shuffle_index]
train_text = train_text[shuffle_index]
trainY = trainY[shuffle_index]

In [ ]:
len(train_images),len(train_text),len(trainY),len(test_images),len(test_text),len(testY)

(4142, 4142, 4142, 1126, 1126, 1126)

In [ ]:
# Load feature file which is a dictionary with image_id as key and feature vector as value. The feature vector is created using bottom up attention.
dict_feat = pickle.load(open("/content/drive/My Drive/multi-modal/weibo/airsplay/dict_feat.pkl", "rb"))

In [ ]:
# Find images for which feature vector size has a mismatch (the total number of fragments are not 36).
c=0
shape_not_match = []
for key in dict_feat :
  if(dict_feat[key].shape[0]!=36) :
    shape_not_match.append(key)
    c+=1
print(c)
print(len(shape_not_match))

677
677


In [ ]:
def index_to_delete_shape(images, shape_not_match) :
  index = []
  for i in range(len(images)) :
    main_image = images[i]
    if ((main_image[:-4] in shape_not_match)) :
      index.append(i)
  return index

In [ ]:
train_delete_index =index_to_delete_shape(train_images, shape_not_match)
test_delete_index = index_to_delete_shape(test_images, shape_not_match)

677

In [ ]:
# Keep only those images for which the feature vector has 36 fragments 
train_images1 = np.delete(train_images,train_delete_index)
train_text1 = np.delete(train_text,train_delete_index)
trainY1 = np.delete(trainY,train_delete_index)
test_images1 = np.delete(test_images,test_delete_index)
test_text1 = np.delete(test_text,test_delete_index)
testY1 = np.delete(testY,test_delete_index)

In [ ]:
len(train_images1),len(train_text1),len(trainY1),len(test_images1),len(test_text1),len(testY1)

(3618, 3618, 3618, 973, 973, 973)

In [ ]:
train_input_img = torch.zeros((3618,36,2048))
train_output_img = torch.zeros((3618,2))

for i in range(len(train_images1)) :
  main_image = train_images1[i]
  train_input_img[i] = dict_feat[main_image[:-4]]
  if (trainY1[i]==0) :
    train_output_img[i][0] = 1
  else :
    train_output_img[i][1] = 1 

In [ ]:
test_input_img = torch.zeros((973,36,2048))
test_output_img = torch.zeros((973,2))

for i in range(len(test_images1)) :
  main_image = test_images1[i]
  test_input_img[i] = dict_feat[main_image[:-4]]
  if (testY1[i]==0) :
    test_output_img[i][0] = 1
  else :
    test_output_img[i][1] = 1

In [ ]:
print(len(train_input_img))
print(len(train_output_img))

3618
3618


## Image Model

In [ ]:
# Code for this cell is from https://github.com/yiling2018/saem/blob/master/bert.py
def gelu(x):
    """Implementation of the gelu activation function.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class BERTLayerNorm(nn.Module):
    def __init__(self, config, variance_epsilon=1e-12):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(BERTLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(config.hidden_size))
        self.beta = nn.Parameter(torch.zeros(config.hidden_size))
        self.variance_epsilon = variance_epsilon

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

class BERTSelfAttention(nn.Module):
    def __init__(self, config):
        super(BERTSelfAttention, self).__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads))
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
        attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        return context_layer


class BERTSelfOutput(nn.Module):
    def __init__(self, config):
        super(BERTSelfOutput, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BERTLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


class BERTAttention(nn.Module):
    def __init__(self, config):
        super(BERTAttention, self).__init__()
        self.self = BERTSelfAttention(config)
        self.output = BERTSelfOutput(config)

    def forward(self, input_tensor, attention_mask):
        self_output = self.self(input_tensor, attention_mask)
        attention_output = self.output(self_output, input_tensor)
        return attention_output


class BERTIntermediate(nn.Module):
    def __init__(self, config):
        super(BERTIntermediate, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        self.intermediate_act_fn = gelu

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states


class BERTOutput(nn.Module):
    def __init__(self, config):
        super(BERTOutput, self).__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = BERTLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


class BERTLayer(nn.Module):
    def __init__(self, config):
        super(BERTLayer, self).__init__()
        self.attention = BERTAttention(config)
        self.intermediate = BERTIntermediate(config)
        self.output = BERTOutput(config)

    def forward(self, hidden_states, attention_mask):
        attention_output = self.attention(hidden_states, attention_mask)
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

In [ ]:
class TransformerMapping(nn.Module):
    """ Self-attention layer for image branch
    """
    def __init__(self):
        super(TransformerMapping, self).__init__()
        bert_config = BertConfig.from_json_file("t_cfg_weibo.json")
        self.layer = BERTLayer(bert_config)
        self.mapping = nn.Linear(2048, 128)
        self.dropout1 = nn.Dropout(0.4)
        self.cls_layer = nn.Linear(128, 2)

    def forward(self, x):
        x = self.mapping(x) # x: (batch_size, patch_num, final_dims)
        attention_mask = torch.ones(x.size(0), x.size(1))
        if torch.cuda.is_available():
            attention_mask = attention_mask.cuda()
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.float()
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        hidden_states = self.layer(x, extended_attention_mask)
        embed = torch.mean(hidden_states, 1) # (batch_size, final_dims)
        codes = F.normalize(embed, p=2, dim=1)  # (N, C)
        codes = self.dropout1(codes)
        codes = self.cls_layer(codes)
        codes = F.softmax(codes, dim=1)
        return codes

## Text embedding model

In [ ]:
def freeze_layers(model):
    for child in model.children():
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
class BertMapping(nn.Module):
    """
    """
    def __init__(self):
        super(BertMapping, self).__init__()
        bert_config = BertConfig.from_pretrained('bert-base-chinese')
        self.bert = BertModel(bert_config)
        freeze_layers(self.bert)
        final_dims = 128
        Ks = [1, 2, 3]
        in_channel = 1
        out_channel = 512
        embedding_dim = bert_config.hidden_size
        self.convs1 = nn.ModuleList([nn.Conv2d(in_channel, out_channel, (K, embedding_dim)) for K in Ks])
        self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        self.mapping = nn.Linear(len(Ks)*out_channel, final_dims)
        self.dropout1 = nn.Dropout(0.4)
        self.cls_layer = nn.Linear(final_dims, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids,attention_mask=attention_mask, return_dict=True)
        x = outputs.last_hidden_state.unsqueeze(1)  # (batch_size, 1, token_num, embedding_dim)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(batch_size, out_channel, W), ...]*len(Ks)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        output = torch.cat(x, 1)

        output = self.dropout(output)
        code = self.mapping(output)
        # code = F.tanh(code)
        code = F.normalize(code, p=2, dim=1)
        code = self.dropout1(code)
        code = self.cls_layer(code)
        code = F.softmax(code, dim=1)
        return code

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

In [ ]:
def get_token_ids(x_train, x_test):
    
    token_tr = []
    token_tst = []
    count = 0
    for sent in x_train :
        tokens = tokenizer.encode(sent, add_special_tokens = True, max_length=512)
        token_tr.append(tokens)
        count+=1
        if(count%1000==0):
            print(count)
    
    for sent1 in x_test :
        tokens1 = tokenizer.encode(sent1, add_special_tokens = True, max_length=512)
        token_tst.append(tokens1)
        count+=1
        if(count%1000==0):
            print(count)
            
    return token_tr, token_tst 

In [ ]:
xtr_token, xtst_token = get_token_ids(train_text1, test_text1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1000
2000
3000
4000


In [ ]:
xtr_token = pad_sequences(xtr_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")
xtst_token = pad_sequences(xtst_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")

In [ ]:
attention_mask_tr = []
attention_mask_tst = []
for sent in xtr_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tr.append(att_mask)

for sent in xtst_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tst.append(att_mask)

In [ ]:
train_input_text = torch.tensor(xtr_token)
test_input_text = torch.tensor(xtst_token)
train_mask = torch.tensor(attention_mask_tr)
test_mask = torch.tensor(attention_mask_tst)

## Final Model

In [ ]:
class FinalModel(nn.Module) :
  """
  """
  def __init__(self, beta):
    super(FinalModel, self).__init__()
    self.text_enc_model = BertMapping()
    self.img_enc_model = TransformerMapping()
    self.beta = beta

  def cal_coeff(self, img_prob, text_prob) :
    logp = torch.log(img_prob)
    logp2 = torch.log(text_prob)
    one_p = 1 - img_prob  # Don't compute gradient here
    one_p.detach()
    one_p2 = 1 - text_prob # Don't compute gradient here
    one_p2.detach()
    # M = 2
    # Beta = 0.5
    coeff = torch.pow(one_p2, self.beta) * logp + torch.pow(one_p, self.beta) * logp2
    return coeff


  def forward(self, input_ids, attention_mask, img) :
    img_enc = self.img_enc_model(img)
    text_enc = self.text_enc_model(input_ids, attention_mask)
    coeff = self.cal_coeff(img_enc, text_enc)
    return coeff

## Training code

In [ ]:
weights = [0.0003429355281207133, 0.0014265335235378032]
sample_weights = [weights[t.argmax().int()] for t in train_output_img]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.sampler import WeightedRandomSampler
batch_size = 16

train_data = TensorDataset(train_input_text, train_mask, train_input_img,train_output_img)
# train_sampler = WeightedRandomSampler(weights=sample_weights, num_samples=3617)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
test_data = TensorDataset(test_input_text, test_mask, test_input_img, test_output_img)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

def eval_model(model, val_loader) :
  model.eval()
  with torch.no_grad() :
    final_out = []
    final_lab = []
    loss_val = 0
    for idx, (val_input, val_mask, val_img, val_label) in enumerate(val_loader):
        val_input = val_input.cuda()
        val_mask = val_mask.cuda()
        val_img = val_img.cuda()
        val_label = val_label.cuda()
        coeff = model(val_input, val_mask, val_img)
        loss = -1*torch.mean(torch.sum(coeff*val_label,dim=1))
        loss_val+= float(loss)
        output = torch.argmax(coeff, dim=1)
        val_label = torch.argmax(val_label, dim=1)
        output = output.cpu().detach().numpy()
        val_label = val_label.cpu().detach().numpy()
        final_out.extend(list(output))
        final_lab.extend(list(val_label))

        del val_input
        del val_label
        del output
        del coeff
        del loss
        del val_mask
        torch.cuda.empty_cache()

  return classification_report(final_lab, final_out, output_dict=True), loss_val

In [ ]:
def train(net, opti, train_loader, num_epochs, val_loader, beta, best_f1_val):
  loss_train = []
  loss_test = []
  best_f1 = best_f1_val
  best_epoch = 0
  for epoch in range(num_epochs):
    loss_val = 0
    for it, (text, mask,img, labels) in enumerate(train_loader):
        opti.zero_grad()  
        text, mask, img, labels = text.cuda(), mask.cuda(), img.cuda(), labels.cuda()
        coeff = net(text, mask, img)
        mul_out = coeff*labels
        sum_out = torch.sum(mul_out, dim=1)
        loss = -1*torch.mean(sum_out)
        loss_val += float(loss.data)
        loss.backward()
        opti.step()

        if (it + 1) % 50 == 0:
            print("Iteration {} of epoch {} complete. Loss : {} ".format(it+1, epoch+1, loss.item()))

        del text
        del mask
        del img
        del labels
        del loss
        del mul_out
        del sum_out
        torch.cuda.empty_cache()

    print('Epoch [{}/{}], Loss:{:.4f}'.format(epoch+1, num_epochs, loss_val))
    loss_train.append(loss_val)
    if((epoch+1)%1==0) :
      report, loss_t= eval_model(net, val_loader)
      f1 = report['macro avg']['f1-score']
      loss_test.append(loss_t)
      print("loss_test", loss_t, "beta", beta)
      print("classification_report")
      print(report)
      if (f1>best_f1) :
            print("best_f1_changed from " + str(best_f1) + " to " + str(f1))
            best_f1 = f1
            best_epoch = epoch
    print("--------------------------------------------------------------")
  return loss_train, loss_test, best_epoch

In [ ]:
for beta in [0.2] : #Add more beta values in list to fine tune
  net = FinalModel(beta).to(device)
  opti = optim.Adam(net.parameters(), lr = 1e-4)
  loss_train, loss_test, be = train(net, opti, train_dataloader, 30, test_dataloader, beta, 0)
  plt.figure()
  plt.plot(loss_train, label="loss_train")
  plt.plot(loss_test, label="loss_test")
  plt.plot([be,be], [0, max(loss_train)])
  plt.legend()
  plt.show()